In [5]:
library(GEOquery) 
library(tidyverse)
library(stringr)
library(shiny)

In [6]:
all_data = getGEO("GSE21935") 
all_data = all_data$GSE21935_series_matrix.txt.gz 

pheno = all_data@phenoData@data 
assay = assayDataElement(all_data@assayData, 'exprs') 

Found 1 file(s)

GSE21935_series_matrix.txt.gz

Using locally cached version: /var/folders/gl/9vxj9q9x0f1b0427vj5rbynm0000gn/T//Rtmp6a5a4Z/GSE21935_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Using locally cached version of GPL570 found here:
/var/folders/gl/9vxj9q9x0f1b0427vj5rbynm0000gn/T//Rtmp6a5a4Z/GPL570.soft 

Warning message:
“62 parsing failures.
  row     col           expected    actual         file
54614 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
54615 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
54616 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
54617 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
54618 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
..... ....... .................. ......... ............
See problems(...) for more details.
”


In [7]:
x <- as.data.frame(t(head(assay, 10))) 


# extract names of genes
gene_names <- colnames(x)

In [8]:
ui <- fluidPage(
  # Application title
  titlePanel("Visualisation of Gene expression data"),
  
  # Sidebar layout with input and output definitions ---
  sidebarLayout(
    # Sidebar with a slider and selection inputs
    sidebarPanel(
      selectInput("selection", "Choose a gene:",
                  choices = gene_names),
    ),
    
    # Main panel for displaying output
    mainPanel(
      # Output: Density plots
      plotOutput(outputId = "plot")
    )
  )
)


In [9]:
y <-  rep("NA", dim(x)[1])

# creating a for loop to assign 0 to control and 1 to scz
for (i in 1:(dim(x)[1])) {
  if(pheno$characteristics_ch1.4[i]=="disease state: control"){
    y[i] = 0
  } else {
    y[i] = 1
  }  
}

In [10]:
# combining data from x and y so that it is easier to specify conditions
z <- cbind(x,y)

In [11]:
# Define server logic required to draw a wordcloud ----
server <- function(input, output) {
  
  kde_plots <- function(selection){
    gene_num <- grep(selection, gene_names)
    kde_left <- density(z[z$y == "0",gene_num])
    kde_right <- density(z[z$y == "1",gene_num])
    return(list(kde_left,kde_right))
  }
  
  output$plot <- renderPlot({
    par(mfrow=c(2,1))
    plot(kde_plots(input$selection)[[1]], main = "Kernel density plot for control group")
    plot(kde_plots(input$selection)[[2]], main = "Kernel density plot for schizophrenic group")
  })
}

In [ ]:
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:3673

